
from discussion - https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021 ( **Power of mean to prob leaderboard** )

with Equal probability 1/6: **LB 1.09**

With Overlapping **LB: 1.1**
```python
seizure_vote  0.196002,
gpd_vote  0.156386,
lrda_vote 0.155805,
other_vote 0.17610,
grda_vote 0.17660,
lpd_vote 0.139101
```
## Conclusion: Test set have means closer to Train Set

with non-overlapping based on spectrogram id
**LB: 1.0**
```python
seizure_vote    0.174031
lpd_vote        0.112700
gpd_vote        0.090854
lrda_vote       0.071484
grda_vote       0.136408
other_vote      0.414523
```
with non-overlapping based on eeg id
**LB: 0.97**
```python
seizure_vote    0.152810
lpd_vote        0.142456
gpd_vote        0.104062
lrda_vote       0.065407
grda_vote       0.114851
other_vote      0.420414
```

## Conclusion: Test set have non-overlapping egg based sequences
with non-overlapping based on p id
**LB: 1.28**
```python
seizure_vote 0.310718
lpd_vote 0.046279
gpd_vote 0.051885
lrda_vote 0.081796
grda_vote 0.231471
other_vote 0.277851
```

## Conclusion: Test set have multiple patient_id sequences

# Overall : Test set have repeat patients with "non overlapping eeg".


### **Most of the code copied from https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-8#Create-Non-Overlapping-Eeg-Id-Train-Data upvode @cdeotte notebook**

## Brain activity notebook series

### [EEGS 10–20 system](https://www.kaggle.com/code/seshurajup/eegs-10-20-system)
Better understanding eegs 10-20 system
### [Missing Eeg_ids Train.csv vs train_eegs [Resolved]](https://www.kaggle.com/code/seshurajup/missing-eeg-ids-in-train-csv-vs-train-eegs-parquet)
Extra training eggs [Resolved] as we can ignore it
### [EDA train.csv](https://www.kaggle.com/code/seshurajup/eda-train-csv)
Detailed analysis of the train.csv
### [Eegs Pairing Analysis & Features](https://www.kaggle.com/code/seshurajup/eegs-pairing-analysis-features)
Pairing features analysis and build features
### [Eegs Target Analysis - Correct way to merge target](https://www.kaggle.com/code/seshurajup/eegs-target-analysis-correct-way-to-merge-target)
How to choice the target votes for training
### [Eegs Train Split (CV)](https://www.kaggle.com/seshurajup/eegs-train-splits-cv)
generate better train split without patient_id overlap

#### **Upvote my work if it is useful**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import StratifiedGroupKFold

sns.set_style("whitegrid")

df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
TARGETS = df.columns[-6:]
str(list(TARGETS))

In [ ]:
train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head(5)

In [ ]:
train.shape

In [ ]:
def calculate_balance_score(distribution):
    flat_distribution = distribution.values.flatten()
    mean_distribution = np.mean(flat_distribution)
    balance_score = mean_absolute_error(flat_distribution, [mean_distribution] * len(flat_distribution))
    return balance_score
from sklearn.model_selection import StratifiedGroupKFold
n_splits = 5
sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=2024)
for fold, (train_idx, val_idx) in enumerate(sgkf.split(train, train['target'], groups=train['patient_id'])):
    train_fold = train.iloc[train_idx]
    val_fold = train.iloc[val_idx]
    train.loc[val_idx,'fold'] = fold
fold_distributions = train.groupby('fold')['target'].value_counts(normalize=True).unstack().fillna(0)
balance_score = calculate_balance_score(fold_distributions)
balance_score

In [ ]:
palette = sns.color_palette("husl", n_colors=len(train['target'].unique()))
n_splits = 5
sgkf = GroupKFold(n_splits=n_splits)
for fold, (train_idx, val_idx) in enumerate(sgkf.split(train, train['target'], groups=train['patient_id'])):
    train_fold = train.iloc[train_idx]
    val_fold = train.iloc[val_idx]
    train.loc[val_idx,'fold'] = fold
    
plt.figure(figsize=(14, 4))
overall_target_order = train['target'].value_counts().sort_index().index
sns.countplot(x='target', data=train, palette=palette, order=overall_target_order)
plt.title('Overall Target Distribution')
plt.show()

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()
for i in range(n_splits):
    fold_data = train[train['fold'] == i]
    if not fold_data.empty:
        target_order = fold_data['target'].value_counts().sort_index().index
        sns.countplot(x='target', data=fold_data, ax=axes[i], palette=palette, order=target_order)
        axes[i].set_title(f'Fold {i} Target Distribution')
    else:
        axes[i].set_title('')
        axes[i].set_visible(False)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

In [ ]:
best_distribution = None
best_balance_score = float('inf')
best_iteration = -1

iterations = 100
for iteration in tqdm(range(iterations), total=iterations):
    shuffled_patient_ids = train['patient_id'].sample(frac=1).reset_index(drop=True)
    train['shuffled_patient_id'] = shuffled_patient_ids

    for fold, (train_idx, val_idx) in enumerate(sgkf.split(train, train['target'], groups=train['shuffled_patient_id'])):
        train.loc[val_idx, 'fold'] = fold

    fold_distributions = train.groupby('fold')['target'].value_counts(normalize=True).unstack().fillna(0)
    balance_score = calculate_balance_score(fold_distributions)

    if balance_score < best_balance_score:
        best_balance_score = balance_score
        best_distribution = fold_distributions
        best_iteration = iteration
        print("Improved", best_balance_score)

best_distribution, best_iteration, best_balance_score

In [ ]:
plt.figure(figsize=(14, 4))
overall_target_order = train['target'].value_counts().sort_index().index
sns.countplot(x='target', data=train, palette=palette, order=overall_target_order)
plt.title('Target Distribution')
plt.show()

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()
palette = sns.color_palette("husl", n_colors=len(train['target'].unique()))
for i in range(n_splits):
    fold_data = train[train['fold'] == i]
    if not fold_data.empty:
        target_order = fold_data['target'].value_counts().sort_index().index
        sns.countplot(x='target', data=fold_data, ax=axes[i], palette=palette, order=target_order)
        axes[i].set_title(f'Fold {i} Target Distribution')
    else:
        axes[i].set_title(f'')
        axes[i].set_visible(False)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

In [ ]:
palette = sns.color_palette("husl", n_colors=len(train['target'].unique()))
n_splits = 5
sgkf = GroupKFold(n_splits=n_splits)
for fold, (train_idx, val_idx) in enumerate(sgkf.split(train, train['target'], groups=train['patient_id'])):
    train_fold = train.iloc[train_idx]
    val_fold = train.iloc[val_idx]
    train.loc[val_idx,'fold'] = fold
    
plt.figure(figsize=(14, 4))
overall_target_order = train['target'].value_counts().sort_index().index
sns.countplot(x='target', data=train, palette=palette, order=overall_target_order)
plt.title('Overall Target Distribution')
plt.show()

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()
for i in range(n_splits):
    fold_data = train[train['fold'] == i]
    if not fold_data.empty:
        target_order = fold_data['target'].value_counts().sort_index().index
        sns.countplot(x='target', data=fold_data, ax=axes[i], palette=palette, order=target_order)
        axes[i].set_title(f'Fold {i} Target Distribution')
    else:
        axes[i].set_title('')
        axes[i].set_visible(False)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

In [ ]:
best_distribution = None
best_balance_score = float('inf')
best_iteration = -1

iterations = len(train)*10
for iteration in tqdm(range(iterations), total=iterations):
    shuffled_patient_ids = train['patient_id'].sample(frac=1).reset_index(drop=True)
    train['shuffled_patient_id'] = shuffled_patient_ids

    for fold, (train_idx, val_idx) in enumerate(sgkf.split(train, train['target'], groups=train['shuffled_patient_id'])):
        train.loc[val_idx, 'fold'] = fold

    fold_distributions = train.groupby('fold')['target'].value_counts(normalize=True).unstack().fillna(0)
    balance_score = calculate_balance_score(fold_distributions)

    if balance_score < best_balance_score:
        best_balance_score = balance_score
        best_distribution = fold_distributions
        best_iteration = iteration
        print("Improved", best_balance_score)

best_distribution, best_iteration, best_balance_score

In [ ]:
best_balance_score

In [ ]:
plt.figure(figsize=(14, 4))
overall_target_order = train['target'].value_counts().sort_index().index
sns.countplot(x='target', data=train, palette=palette, order=overall_target_order)
plt.title('Target Distribution')
plt.show()

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()
palette = sns.color_palette("husl", n_colors=len(train['target'].unique()))
for i in range(n_splits):
    fold_data = train[train['fold'] == i]
    if not fold_data.empty:
        target_order = fold_data['target'].value_counts().sort_index().index
        sns.countplot(x='target', data=fold_data, ax=axes[i], palette=palette, order=target_order)
        axes[i].set_title(f'Fold {i} Target Distribution')
    else:
        axes[i].set_title(f'')
        axes[i].set_visible(False)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])
plt.tight_layout()
plt.show()

# Now split distributions much better 

In [ ]:
train

In [ ]:
del train['shuffled_patient_id']

In [ ]:
train.to_csv("train_splits.csv", index=False)

**Generally I use Stratified Group KFold in the following two situations:**

**one target class is very rare and we want to make sure to include this target class in both train and valid (for each K fold split)
the test data has the same proportion of target classes as train data
In this competition, we are not sure if the test data has the same proportions (of 6 target classes) as train data. Therefore if we use Group KFold (instead of Stratified Group KFold) then we are evaluating how well our model can perform when the test data may have slightly different proportions as train data.**

**If we use Stratified Group KFold, then we are optimizing a model to make predictions on test data that has the same proportion of target classes as train data. In conclusion, they will both work well, but perhaps Group KFold will produce a model that generalizes slightly better to an unknown test proportion.**

- by [@cdeotte](https://www.kaggle.com/cdeotte) in [Discussion CatBoost Starter Notebook and Kaggle Dataset - LB 0.67 - CV 0.82](https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467576#2600727)